In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import random
import os
import pandas as pd
from fractions import Fraction

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


model = "first"
model = "best"

mode = "easy"
mode = "medium"
mode = "difficult"
mode = "all"

phase = "0"
phase = "1"
phase = "2"
phase = "0,1,2"

top_n = 28

impute_value = -2
os.chdir("/home2/s439906/project/CASP16/monomer/")
path = "./monomer_data_newest/processed/"

metadata_file = "./monomer_EU_info"
EU_weight = {}
with open(metadata_file, "r") as f:
    for line in f:
        if line.startswith("T"):
            words = line.strip().split()
            protein = words[0]
            if protein == "T1249":
                protein = "T1249v1"
            if protein == "T2249":
                protein = "T2249v1"
            domain = words[1]
            difficulty = words[2]
            weight = float(Fraction(words[4]))
            EU = f"{protein}-{domain}"
            EU_weight[EU] = weight

difficulty_group = {"easy": [], "medium": [], "difficult": []}
difficulty_file = "./step7_result"
with open(difficulty_file, "r") as f:
    for line in f:
        if line.startswith("T"):
            words = line.strip().split()
            protein = words[0]
            if protein == "T1249-D1":
                protein = "T1249v1-D1"
            if protein == "T2249-D1":
                protein = "T2249v1-D1"
            difficulty = words[2]
            EU = protein
            if difficulty == "easy":
                difficulty_group[difficulty].append(EU)
            elif difficulty == "medium":
                difficulty_group[difficulty].append(EU)
            elif difficulty == "hard":
                difficulty_group["difficult"].append(EU)

files = [file for file in os.listdir(path) if file.startswith("T") and file.endswith(".csv")]
files.sort()

if mode == "easy":
    files = [file for file in files if file[:-4] in difficulty_group["easy"]]
elif mode == "medium":
    files = [file for file in files if file[:-4] in difficulty_group["medium"]]
elif mode == "difficult":
    files = [file for file in files if file[:-4] in difficulty_group["difficult"]]
elif mode == "all":
    pass
if phase == "0":
    files = [file for file in files if file.startswith("T0")]
elif phase == "1":
    files = [file for file in files if file.startswith("T1")]
elif phase == "2":
    files = [file for file in files if file.startswith("T2")]
elif phase == "0,1,2":
    pass

df = pd.DataFrame()
for file in files:
    df_tmp = pd.read_csv(path + file, index_col=0)
    df = pd.concat([df, df_tmp])
df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?').apply(lambda x: (f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0], f"TS{x[1]}", x[2]), axis=1)

df = pd.DataFrame()
for file in files:
    df_tmp = pd.read_csv(path + file, index_col=0)
    df = pd.concat([df, df_tmp])
df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?').apply(lambda x: (f"TS{x[1]}", f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0],  x[2]), axis=1)
# sort index
df = df.sort_index()
df.index = pd.MultiIndex.from_tuples(
            df.index, names=['group', 'target', 'model'])



df = pd.DataFrame()

for file in files:
    df_tmp = pd.read_csv(os.path.join(path, file), index_col=0)
    df = pd.concat([df, df_tmp])

df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?'
).apply(lambda x: (f"TS{x[1]}", f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0], x[2]), axis=1)

df = df.sort_index()

df.index = pd.MultiIndex.from_tuples(
    df.index, names=['group', 'target', 'model']
)

group_target_counts = df.index.get_level_values('group').value_counts()
len(files)

In [ ]:
# do bootstrap
weights = [
                        1/6, 1/16, 1/6,
                        1/6,
                        1/16, 1/8,
                        1/8, 1/16,
                        1/16,
                    ]
group_dict = {}
for file in files:
    EU = file.split('.')[0]
    with open(path + file, 'r') as f:
        for line in f:
            if line.startswith("T"):
                word = line.strip().split(',')
                meta_data = word[0]
                _, meta_data = meta_data.split('TS')
                group, model_id = meta_data.split('_')
                group = 'TS' + group
                model_id = model_id[0]
                if group not in group_dict:
                    group_dict[group] = {}
                if EU not in group_dict[group]:
                    group_dict[group][EU] = {}
                score = sum([float(word[i+1] )* weights[i] for i in range(9)]) * EU_weight[EU] 
                if model_id not in group_dict[group][EU]:
                    group_dict[group][EU][model_id] = score
                else:
                    group_dict[group][EU][model_id] = max(score, group_dict[group][EU][model_id])
    # if there is no model_1, replace the model with the smallest id to be model_1
    for group in group_dict:
        for EU in group_dict[group]:
            if '1' not in group_dict[group][EU]:
                min_model = min(group_dict[group][EU], key=group_dict[group][EU].get)
                group_dict[group][EU]['1'] = group_dict[group][EU][min_model]
                del group_dict[group][EU][min_model]
 


group_dict = dict(sorted(group_dict.items(), key=lambda x: x[0]))
group_dict
score_path = "./score_all/"
ranking_file = f"sum-{model}-{mode}-{phase}-impute={impute_value}-EU_weighted_sum.csv"
ranking_info = pd.read_csv(score_path + ranking_file, index_col=0)
ranking_info
top_n_list = ranking_info.index[:top_n].tolist()
top_n_list
group_dict = {k: v for k, v in group_dict.items() if k in top_n_list}
assert len(group_dict) == top_n

bootstrap_dict  = {}
for group_1 in group_dict.keys():
    bootstrap_dict[group_1] = {}
    for group_2 in group_dict.keys():
        bootstrap_dict[group_1][group_2] = 0
    
bootstrap_round = 1000
for i in range(bootstrap_round):
    for group_1, group_2 in combinations(group_dict.keys(), 2):
        data_1 = group_dict[group_1]
        data_2 = group_dict[group_2]
        common_EU = set(data_1.keys()) & set(data_2.keys())
        sampled_EU = random.choices(list(common_EU), k=len(common_EU))
        score_1 = 0
        score_2 = 0
        for eu in sampled_EU:
            value_dict_1 = data_1[eu]
            value_dict_2 = data_2[eu]
            # print(value_list_1)
            # print(value_list_2)
            # sampled_value_1 = random.choice(value_list_1)
            # sampled_value_2 = random.choice(value_list_2)
            # sampled_value_1 =  max(value_dict_1.values())
            # sampled_value_2 =  max(value_dict_2.values())
            # sampled_value_1 = value_list_1['1']
            # sampled_value_2 = value_list_2['1']
            if model == "first":
                sampled_value_1 = value_dict_1['1']
                sampled_value_2 = value_dict_2['1']
            elif model == "best":
                # take the common keys in the two dicts
                common_keys = set(value_dict_1.keys()) & set(value_dict_2.keys())
                # get the best score in the common keys
                sampled_value_1 = max([value_dict_1[key] for key in common_keys])
                sampled_value_2 = max([value_dict_2[key] for key in common_keys])
            score_1 += sampled_value_1
            score_2 += sampled_value_2
        if score_1 > score_2:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 1
        elif score_1 < score_2:
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 1
        else:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 0.5
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 0.5

    if i % 100 == 99:
        print(f"Bootstrap {i+1} done.")
bootstrap_dict

In [ ]:
# get bootstrap rank
for group_1 in bootstrap_dict:
    for group_2 in bootstrap_dict[group_1]:
        bootstrap_dict[group_1][group_2] /= bootstrap_round
bootstrap_sum = {}
for group_1 in bootstrap_dict:
    for group_2 in bootstrap_dict[group_1]:
        if group_1 not in bootstrap_sum:
            bootstrap_sum[group_1] = 0
        bootstrap_sum[group_1] += bootstrap_dict[group_1][group_2]
        
bootstrap_sum = dict(sorted(bootstrap_sum.items(), key=lambda x: x[1], reverse=True))
bootstrap_sum

In [ ]:
# plot

bootstrap_rank = [key for key in bootstrap_sum]
bootstrap_dict = dict(sorted(bootstrap_dict.items(), key=lambda x: bootstrap_sum[x[0]], reverse=True))
for group in bootstrap_dict:
    bootstrap_dict[group] = dict(sorted(bootstrap_dict[group].items(), key=lambda x: bootstrap_sum[x[0]],reverse=True))
bootstrap_array = np.zeros((top_n, top_n))
for i, group_1 in enumerate(bootstrap_dict):
    for j, group_2 in enumerate(bootstrap_dict[group_1]):
        bootstrap_array[i, j] = bootstrap_dict[group_1][group_2]
bootstrap_array,bootstrap_rank



plt.figure(figsize=(16, 12))
ax = sns.heatmap(bootstrap_array, annot=True, fmt=".2f",
                 cmap='Greys', cbar=True, square=True,
                 #   linewidths=1, linecolor='black',
                 )

for text in ax.texts:
    value = float(text.get_text())
    if value >= 0.95:
        text.set_color('red')
    elif value < 0.95 and value >= 0.75:
        text.set_color('white')
    else:
        text.set_color('black')

cbar = ax.collections[0].colorbar
cbar.set_ticks([0, float(1/4), float(1/2),
                float(3/4), 1])
cbar.set_ticklabels([0, float(1/4), float(1/2),
                    float(3/4), 1])
cbar.ax.tick_params(labelsize=12)
for _, spine in ax.spines.items():
    spine.set_visible(True)
    spine.set_linewidth(2)
ax.set_xticklabels(ax.get_xticklabels(), horizontalalignment='center')
ax.set_yticklabels(ax.get_yticklabels(), verticalalignment='center')
plt.xticks(np.arange(top_n), bootstrap_rank,rotation=90, fontsize=18)
plt.yticks(np.arange(top_n), bootstrap_rank, rotation=0, fontsize=18)
plt.title(f"models: {model}, targets: {mode}, top_n: {top_n}, phase: {phase}, impute: {impute_value}", fontsize=20, pad=30)
fig_path = "./CASP16_figures/"
png_top_file = f"bootstrap_{model}_{mode}_{phase}_impute={impute_value}_top_{top_n}.png"
plt.savefig(fig_path + png_top_file, dpi=300)
plt.show()